In [1]:
from typing import Dict
import requests
import json
import pandas as pd
from deepchecks.tabular.suites import full_suite
from deepchecks.tabular.suites import data_integrity
from deepchecks.tabular import Dataset

api_url = "http://192.168.2.30:3002/api/"

def get_latest_block() -> int:
    """
    Retrieves the height of the latest block from a specified API endpoint.

    Args:
        api_url (str): The URL of the API endpoint.

    Returns:
        int: The height of the latest block as an integer.
    """
    response = requests.get(api_url + "blocks/tip")
    response_json = response.json()
    return int(response_json["height"])


def get_block_info(block_height: int) -> dict:
    """
    Retrieves information about a specific block from a given API endpoint.

    Args:
        block_height (int): The height of the block for which information is requested.

    Returns:
        dict: A JSON object containing the information about the requested block.
    """
    url = f"{api_url}block/{block_height}"
    response = requests.get(url)
    return json.loads(response.text)

def get_transaction(tx):
    url = f"{api_url}tx/{tx}"
    response = requests.get(url)
    return json.loads(response.text)

latest_block = get_latest_block()

n_blocks = 2

tx_df = []
for i in range(latest_block - 1, latest_block - (n_blocks + 1), -1):
    print(i)
    txs = get_block_info(i)['tx']

    for tx in txs[0:1]:
        tx_json = get_transaction(tx)
        normalized_df = pd.DataFrame(pd.json_normalize(tx_json))
        
        for column in normalized_df.columns:
            if normalized_df[column].apply(lambda x: isinstance(x, list)).any():
                normalized_df = normalized_df.explode(column)

        tx_df.append(normalized_df)
    
df = pd.concat(tx_df)

print(df)

ds_train = Dataset(df, cat_features=[])

integ_suite = data_integrity()
integ_suite.run(ds_train)

833439
833438
                                                txid  \
0  4df8aa089f52b925dcc913d38c13638a392633e12ea836...   
0  4df8aa089f52b925dcc913d38c13638a392633e12ea836...   
0  4df8aa089f52b925dcc913d38c13638a392633e12ea836...   
0  e96f4fb9c1a1ab4849a055b96fa016f825f7b5cf5ae7ef...   
0  e96f4fb9c1a1ab4849a055b96fa016f825f7b5cf5ae7ef...   
0  e96f4fb9c1a1ab4849a055b96fa016f825f7b5cf5ae7ef...   
0  e96f4fb9c1a1ab4849a055b96fa016f825f7b5cf5ae7ef...   
0  e96f4fb9c1a1ab4849a055b96fa016f825f7b5cf5ae7ef...   

                                                hash  version  size  vsize  \
0  b39ebdf65c372f705a10eb6876225e369089c109cda862...        1   299    272   
0  b39ebdf65c372f705a10eb6876225e369089c109cda862...        1   299    272   
0  b39ebdf65c372f705a10eb6876225e369089c109cda862...        1   299    272   
0  dc80ada59e6ccb6a9fafe781488330d85cb81af752cb6b...        1   392    365   
0  dc80ada59e6ccb6a9fafe781488330d85cb81af752cb6b...        1   392    365   
0  dc80ada59e

/home/wim/.cache/pypoetry/virtualenvs/bitcoin-data-9VL3xZDu-py3.10/lib/python3.10/site-packages/deepchecks/tabular/dataset.py:236: UserWarning:

Dataframe index has duplicate indexes, setting index to [0,1..,n-1].



Accordion(children=(VBox(children=(HTML(value='\n<h1 id="summary_URGGT8X7B1159T4ORUUU9CI1J">Data Integrity Sui…